In [8]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types

In [52]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [4]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [6]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [9]:
db = DFDB('../trial/catboost.pkl', auto_commit=False)

In [8]:
#check feature_importances
# df_feature_importances = df_trial.loc[235]['df_feature_importances']
# if type(df_feature_importances)==pd.DataFrame:
#     sorted_columns = EP.evaluate(df_feature_importances, key='average_model_weight')
# else:
#     sorted_columns = df_trial.loc[235]['param']['columns']
# df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False)
# len(sorted_columns)

In [11]:
# param = df_trial.loc[342]['param'].copy()
param = {'columns': tsfresh_columns,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 589,
   'depth': 6,
   'learning_rate': 0.05293979792364842,
   'l2_leaf_reg': 78.065140245968,
   'bagging_temperature': 0.9302786271852079,
   'random_strength': 0.4247048326178351,
   'random_state': 651},
  'fit': {}},
}

In [13]:
mytrial =[]
EP.select_features_(df_train, param, mytrial, nfeats_best=10, nfeats_removed_per_try=20, key='average_model_weight', remark='group3 RFE')
for trial_i in mytrial:
    db.insert(trial_i)

0:	learn: 6.3783435	total: 260ms	remaining: 2m 32s
1:	learn: 6.1052608	total: 447ms	remaining: 2m 11s
2:	learn: 5.8472597	total: 627ms	remaining: 2m 2s
3:	learn: 5.6057368	total: 824ms	remaining: 2m
4:	learn: 5.3767407	total: 1.02s	remaining: 1m 59s
5:	learn: 5.1628207	total: 1.2s	remaining: 1m 56s
6:	learn: 4.9627806	total: 1.38s	remaining: 1m 55s
7:	learn: 4.7764407	total: 1.58s	remaining: 1m 55s
8:	learn: 4.6012799	total: 1.77s	remaining: 1m 54s
9:	learn: 4.4383303	total: 1.97s	remaining: 1m 54s
10:	learn: 4.2870399	total: 2.17s	remaining: 1m 53s
11:	learn: 4.1457925	total: 2.36s	remaining: 1m 53s
12:	learn: 4.0138856	total: 2.55s	remaining: 1m 53s
13:	learn: 3.8920130	total: 2.75s	remaining: 1m 52s
14:	learn: 3.7790114	total: 2.93s	remaining: 1m 52s
15:	learn: 3.6737505	total: 3.11s	remaining: 1m 51s
16:	learn: 3.5753273	total: 3.3s	remaining: 1m 51s
17:	learn: 3.4837949	total: 3.5s	remaining: 1m 50s
18:	learn: 3.4012894	total: 3.68s	remaining: 1m 50s
19:	learn: 3.3244214	total: 3.

158:	learn: 2.2794359	total: 29.6s	remaining: 1m 20s
159:	learn: 2.2785531	total: 29.8s	remaining: 1m 19s
160:	learn: 2.2772620	total: 29.9s	remaining: 1m 19s
161:	learn: 2.2758370	total: 30.1s	remaining: 1m 19s
162:	learn: 2.2754313	total: 30.3s	remaining: 1m 19s
163:	learn: 2.2745148	total: 30.5s	remaining: 1m 19s
164:	learn: 2.2737398	total: 30.7s	remaining: 1m 18s
165:	learn: 2.2731658	total: 30.9s	remaining: 1m 18s
166:	learn: 2.2723089	total: 31s	remaining: 1m 18s
167:	learn: 2.2718824	total: 31.2s	remaining: 1m 18s
168:	learn: 2.2707582	total: 31.4s	remaining: 1m 18s
169:	learn: 2.2699637	total: 31.6s	remaining: 1m 17s
170:	learn: 2.2693515	total: 31.8s	remaining: 1m 17s
171:	learn: 2.2689064	total: 31.9s	remaining: 1m 17s
172:	learn: 2.2676810	total: 32.1s	remaining: 1m 17s
173:	learn: 2.2670165	total: 32.3s	remaining: 1m 17s
174:	learn: 2.2668325	total: 32.5s	remaining: 1m 16s
175:	learn: 2.2659806	total: 32.6s	remaining: 1m 16s
176:	learn: 2.2647924	total: 32.8s	remaining: 1m

316:	learn: 2.1549497	total: 57.4s	remaining: 49.3s
317:	learn: 2.1544619	total: 57.6s	remaining: 49.1s
318:	learn: 2.1535978	total: 57.7s	remaining: 48.9s
319:	learn: 2.1531382	total: 57.9s	remaining: 48.7s
320:	learn: 2.1519360	total: 58.1s	remaining: 48.5s
321:	learn: 2.1508718	total: 58.3s	remaining: 48.3s
322:	learn: 2.1495433	total: 58.5s	remaining: 48.1s
323:	learn: 2.1481763	total: 58.6s	remaining: 48s
324:	learn: 2.1466730	total: 58.8s	remaining: 47.8s
325:	learn: 2.1465358	total: 59s	remaining: 47.6s
326:	learn: 2.1457524	total: 59.1s	remaining: 47.4s
327:	learn: 2.1453672	total: 59.3s	remaining: 47.2s
328:	learn: 2.1446946	total: 59.5s	remaining: 47s
329:	learn: 2.1436558	total: 59.7s	remaining: 46.8s
330:	learn: 2.1431502	total: 59.9s	remaining: 46.7s
331:	learn: 2.1427760	total: 1m	remaining: 46.5s
332:	learn: 2.1421524	total: 1m	remaining: 46.3s
333:	learn: 2.1418597	total: 1m	remaining: 46.1s
334:	learn: 2.1415772	total: 1m	remaining: 45.9s
335:	learn: 2.1412665	total: 1

473:	learn: 2.0404835	total: 1m 25s	remaining: 20.6s
474:	learn: 2.0399933	total: 1m 25s	remaining: 20.5s
475:	learn: 2.0392686	total: 1m 25s	remaining: 20.3s
476:	learn: 2.0386890	total: 1m 25s	remaining: 20.1s
477:	learn: 2.0383104	total: 1m 25s	remaining: 19.9s
478:	learn: 2.0377741	total: 1m 25s	remaining: 19.7s
479:	learn: 2.0372858	total: 1m 26s	remaining: 19.6s
480:	learn: 2.0362083	total: 1m 26s	remaining: 19.4s
481:	learn: 2.0360734	total: 1m 26s	remaining: 19.2s
482:	learn: 2.0347902	total: 1m 26s	remaining: 19s
483:	learn: 2.0341941	total: 1m 26s	remaining: 18.8s
484:	learn: 2.0341591	total: 1m 27s	remaining: 18.7s
485:	learn: 2.0330933	total: 1m 27s	remaining: 18.5s
486:	learn: 2.0325001	total: 1m 27s	remaining: 18.3s
487:	learn: 2.0317864	total: 1m 27s	remaining: 18.1s
488:	learn: 2.0317554	total: 1m 27s	remaining: 17.9s
489:	learn: 2.0312703	total: 1m 27s	remaining: 17.8s
490:	learn: 2.0299255	total: 1m 28s	remaining: 17.6s
491:	learn: 2.0294728	total: 1m 28s	remaining: 1

40:	learn: 2.7298174	total: 8.01s	remaining: 1m 47s
41:	learn: 2.7170642	total: 8.19s	remaining: 1m 46s
42:	learn: 2.7053931	total: 8.39s	remaining: 1m 46s
43:	learn: 2.6942080	total: 8.58s	remaining: 1m 46s
44:	learn: 2.6861571	total: 8.79s	remaining: 1m 46s
45:	learn: 2.6757695	total: 8.97s	remaining: 1m 45s
46:	learn: 2.6672121	total: 9.18s	remaining: 1m 45s
47:	learn: 2.6590333	total: 9.38s	remaining: 1m 45s
48:	learn: 2.6532417	total: 9.56s	remaining: 1m 45s
49:	learn: 2.6459784	total: 9.76s	remaining: 1m 45s
50:	learn: 2.6401431	total: 9.95s	remaining: 1m 44s
51:	learn: 2.6342943	total: 10.1s	remaining: 1m 44s
52:	learn: 2.6259997	total: 10.3s	remaining: 1m 44s
53:	learn: 2.6195368	total: 10.5s	remaining: 1m 44s
54:	learn: 2.6140664	total: 10.7s	remaining: 1m 44s
55:	learn: 2.6074193	total: 10.9s	remaining: 1m 43s
56:	learn: 2.6035523	total: 11.1s	remaining: 1m 43s
57:	learn: 2.5986932	total: 11.3s	remaining: 1m 43s
58:	learn: 2.5933935	total: 11.5s	remaining: 1m 43s
59:	learn: 2

...

26:	learn: 2.9263071	total: 5.14s	remaining: 1m 47s
27:	learn: 2.8862389	total: 5.33s	remaining: 1m 46s
28:	learn: 2.8493965	total: 5.52s	remaining: 1m 46s
29:	learn: 2.8160152	total: 5.72s	remaining: 1m 46s
30:	learn: 2.7844420	total: 5.91s	remaining: 1m 46s
31:	learn: 2.7545196	total: 6.09s	remaining: 1m 45s
32:	learn: 2.7269148	total: 6.28s	remaining: 1m 45s
33:	learn: 2.7022944	total: 6.46s	remaining: 1m 45s
34:	learn: 2.6805909	total: 6.64s	remaining: 1m 45s
35:	learn: 2.6596770	total: 6.84s	remaining: 1m 45s
36:	learn: 2.6399874	total: 7.02s	remaining: 1m 44s
37:	learn: 2.6218817	total: 7.2s	remaining: 1m 44s
38:	learn: 2.6042691	total: 7.38s	remaining: 1m 44s
39:	learn: 2.5878500	total: 7.56s	remaining: 1m 43s
40:	learn: 2.5742531	total: 7.73s	remaining: 1m 43s
41:	learn: 2.5607219	total: 7.91s	remaining: 1m 42s
42:	learn: 2.5486757	total: 8.09s	remaining: 1m 42s
43:	learn: 2.5375193	total: 8.27s	remaining: 1m 42s
44:	learn: 2.5270697	total: 8.46s	remaining: 1m 42s
45:	learn: 2.

183:	learn: 2.2513740	total: 31.1s	remaining: 1m 8s
184:	learn: 2.2506573	total: 31.3s	remaining: 1m 8s
185:	learn: 2.2502593	total: 31.5s	remaining: 1m 8s
186:	learn: 2.2493033	total: 31.6s	remaining: 1m 7s
187:	learn: 2.2484303	total: 31.8s	remaining: 1m 7s
188:	learn: 2.2479922	total: 31.9s	remaining: 1m 7s
189:	learn: 2.2467957	total: 32.1s	remaining: 1m 7s
190:	learn: 2.2463236	total: 32.2s	remaining: 1m 7s
191:	learn: 2.2459571	total: 32.3s	remaining: 1m 6s
192:	learn: 2.2453633	total: 32.5s	remaining: 1m 6s
193:	learn: 2.2450451	total: 32.7s	remaining: 1m 6s
194:	learn: 2.2446382	total: 32.8s	remaining: 1m 6s
195:	learn: 2.2440314	total: 33s	remaining: 1m 6s
196:	learn: 2.2433632	total: 33.1s	remaining: 1m 5s
197:	learn: 2.2430852	total: 33.3s	remaining: 1m 5s
198:	learn: 2.2427110	total: 33.4s	remaining: 1m 5s
199:	learn: 2.2422166	total: 33.6s	remaining: 1m 5s
200:	learn: 2.2410772	total: 33.7s	remaining: 1m 5s
201:	learn: 2.2400310	total: 33.9s	remaining: 1m 4s
202:	learn: 2.

342:	learn: 2.1319166	total: 55.1s	remaining: 39.5s
343:	learn: 2.1316206	total: 55.2s	remaining: 39.3s
344:	learn: 2.1309980	total: 55.4s	remaining: 39.2s
345:	learn: 2.1301446	total: 55.5s	remaining: 39s
346:	learn: 2.1293143	total: 55.7s	remaining: 38.8s
347:	learn: 2.1279066	total: 55.8s	remaining: 38.7s
348:	learn: 2.1271160	total: 56s	remaining: 38.5s
349:	learn: 2.1258538	total: 56.1s	remaining: 38.3s
350:	learn: 2.1254456	total: 56.3s	remaining: 38.2s
351:	learn: 2.1247192	total: 56.4s	remaining: 38s
352:	learn: 2.1243606	total: 56.6s	remaining: 37.8s
353:	learn: 2.1239872	total: 56.7s	remaining: 37.7s
354:	learn: 2.1232163	total: 56.9s	remaining: 37.5s
355:	learn: 2.1222214	total: 57s	remaining: 37.3s
356:	learn: 2.1217372	total: 57.2s	remaining: 37.1s
357:	learn: 2.1213863	total: 57.3s	remaining: 37s
358:	learn: 2.1194468	total: 57.5s	remaining: 36.8s
359:	learn: 2.1192145	total: 57.6s	remaining: 36.7s
360:	learn: 2.1180484	total: 57.8s	remaining: 36.5s
361:	learn: 2.1179420	

500:	learn: 2.0137013	total: 1m 18s	remaining: 13.8s
501:	learn: 2.0129839	total: 1m 18s	remaining: 13.7s
502:	learn: 2.0120324	total: 1m 19s	remaining: 13.5s
503:	learn: 2.0111449	total: 1m 19s	remaining: 13.4s
504:	learn: 2.0103886	total: 1m 19s	remaining: 13.2s
505:	learn: 2.0094339	total: 1m 19s	remaining: 13s
506:	learn: 2.0087764	total: 1m 19s	remaining: 12.9s
507:	learn: 2.0078614	total: 1m 19s	remaining: 12.7s
508:	learn: 2.0072963	total: 1m 19s	remaining: 12.6s
509:	learn: 2.0069633	total: 1m 20s	remaining: 12.4s
510:	learn: 2.0067689	total: 1m 20s	remaining: 12.3s
511:	learn: 2.0060347	total: 1m 20s	remaining: 12.1s
512:	learn: 2.0051025	total: 1m 20s	remaining: 11.9s
513:	learn: 2.0038226	total: 1m 20s	remaining: 11.8s
514:	learn: 2.0025709	total: 1m 20s	remaining: 11.6s
515:	learn: 2.0023038	total: 1m 21s	remaining: 11.5s
516:	learn: 2.0013253	total: 1m 21s	remaining: 11.3s
517:	learn: 2.0001847	total: 1m 21s	remaining: 11.2s
518:	learn: 1.9993794	total: 1m 21s	remaining: 1

KeyboardInterrupt: 

In [15]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [16]:
df_trial[['datetime', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-17 19:10:11.182668,1071,1.601879,0.000069,2.121013,0.007367,0.519134
1,2019-05-18 04:01:18.357701,1051,1.598117,0.000113,2.122386,0.007074,0.524270
2,2019-05-18 12:29:57.440833,1031,1.599668,0.000188,2.117059,0.007864,0.517391
3,2019-05-18 20:49:18.314789,1011,1.597300,0.000085,2.120985,0.007927,0.523685


In [36]:
top4_sorted_columns = []
for i in range(4):
    df_feature_importances = df_trial.loc[i]['df_feature_importances']
    sorted_columns = EP.evaluate(df_feature_importances, key='average_model_weight')
    top4_sorted_columns += sorted_columns[:100]

In [39]:
sorted_columns = list(set(top4_sorted_columns))
len(top4_sorted_columns), len(set(top4_sorted_columns))

(400, 134)

In [40]:
# param = df_trial.loc[342]['param'].copy()
param = {'columns': sorted_columns,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 589,
   'depth': 6,
   'learning_rate': 0.05293979792364842,
   'l2_leaf_reg': 78.065140245968,
   'bagging_temperature': 0.9302786271852079,
   'random_strength': 0.4247048326178351,
   'random_state': 651},
  'fit': {}},
}

In [43]:
mytrial =[]
EP.select_features_(df_train, param, mytrial, nfeats_best=10, nfeats_removed_per_try=20, key='average_model_weight', remark='group3 RFE2')
for trial_i in mytrial:
    db.insert(trial_i)

0:	learn: 6.3780292	total: 81.1ms	remaining: 47.7s
1:	learn: 6.1026582	total: 132ms	remaining: 38.8s
2:	learn: 5.8451613	total: 205ms	remaining: 40s
3:	learn: 5.6043348	total: 256ms	remaining: 37.4s
4:	learn: 5.3777766	total: 295ms	remaining: 34.5s
5:	learn: 5.1638180	total: 338ms	remaining: 32.8s
6:	learn: 4.9631143	total: 385ms	remaining: 32s
7:	learn: 4.7769820	total: 430ms	remaining: 31.2s
8:	learn: 4.6012863	total: 478ms	remaining: 30.8s
9:	learn: 4.4378923	total: 526ms	remaining: 30.4s
10:	learn: 4.2858294	total: 565ms	remaining: 29.7s
11:	learn: 4.1437964	total: 602ms	remaining: 29s
12:	learn: 4.0114706	total: 652ms	remaining: 28.9s
13:	learn: 3.8878991	total: 697ms	remaining: 28.6s
14:	learn: 3.7743207	total: 742ms	remaining: 28.4s
15:	learn: 3.6698601	total: 785ms	remaining: 28.1s
16:	learn: 3.5722463	total: 830ms	remaining: 27.9s
17:	learn: 3.4799847	total: 875ms	remaining: 27.8s
18:	learn: 3.3966261	total: 920ms	remaining: 27.6s
19:	learn: 3.3195893	total: 967ms	remaining: 2

161:	learn: 2.2820952	total: 7.44s	remaining: 19.6s
162:	learn: 2.2807865	total: 7.49s	remaining: 19.6s
163:	learn: 2.2801176	total: 7.54s	remaining: 19.5s
164:	learn: 2.2791555	total: 7.58s	remaining: 19.5s
165:	learn: 2.2783527	total: 7.63s	remaining: 19.4s
166:	learn: 2.2779204	total: 7.68s	remaining: 19.4s
167:	learn: 2.2768384	total: 7.73s	remaining: 19.4s
168:	learn: 2.2757969	total: 7.77s	remaining: 19.3s
169:	learn: 2.2751747	total: 7.82s	remaining: 19.3s
170:	learn: 2.2745262	total: 7.87s	remaining: 19.2s
171:	learn: 2.2742267	total: 7.91s	remaining: 19.2s
172:	learn: 2.2733631	total: 7.95s	remaining: 19.1s
173:	learn: 2.2730021	total: 8s	remaining: 19.1s
174:	learn: 2.2722569	total: 8.05s	remaining: 19.1s
175:	learn: 2.2715712	total: 8.1s	remaining: 19s
176:	learn: 2.2708490	total: 8.15s	remaining: 19s
177:	learn: 2.2704189	total: 8.19s	remaining: 18.9s
178:	learn: 2.2691803	total: 8.23s	remaining: 18.9s
179:	learn: 2.2688953	total: 8.28s	remaining: 18.8s
180:	learn: 2.268056

320:	learn: 2.1687757	total: 14.8s	remaining: 12.4s
321:	learn: 2.1684952	total: 14.9s	remaining: 12.3s
322:	learn: 2.1681483	total: 14.9s	remaining: 12.3s
323:	learn: 2.1680557	total: 15s	remaining: 12.3s
324:	learn: 2.1675995	total: 15s	remaining: 12.2s
325:	learn: 2.1672279	total: 15.1s	remaining: 12.2s
326:	learn: 2.1658358	total: 15.1s	remaining: 12.1s
327:	learn: 2.1651616	total: 15.2s	remaining: 12.1s
328:	learn: 2.1640346	total: 15.2s	remaining: 12s
329:	learn: 2.1634036	total: 15.3s	remaining: 12s
330:	learn: 2.1622433	total: 15.3s	remaining: 11.9s
331:	learn: 2.1615972	total: 15.3s	remaining: 11.9s
332:	learn: 2.1603465	total: 15.4s	remaining: 11.8s
333:	learn: 2.1598454	total: 15.4s	remaining: 11.8s
334:	learn: 2.1587877	total: 15.5s	remaining: 11.7s
335:	learn: 2.1585033	total: 15.5s	remaining: 11.7s
336:	learn: 2.1582784	total: 15.6s	remaining: 11.6s
337:	learn: 2.1575200	total: 15.6s	remaining: 11.6s
338:	learn: 2.1571390	total: 15.6s	remaining: 11.5s
339:	learn: 2.156148

479:	learn: 2.0764871	total: 21.9s	remaining: 4.97s
480:	learn: 2.0756459	total: 22s	remaining: 4.93s
481:	learn: 2.0743528	total: 22s	remaining: 4.88s
482:	learn: 2.0737705	total: 22.1s	remaining: 4.84s
483:	learn: 2.0735014	total: 22.1s	remaining: 4.79s
484:	learn: 2.0727819	total: 22.1s	remaining: 4.75s
485:	learn: 2.0723619	total: 22.2s	remaining: 4.7s
486:	learn: 2.0715845	total: 22.2s	remaining: 4.65s
487:	learn: 2.0714299	total: 22.3s	remaining: 4.61s
488:	learn: 2.0712698	total: 22.3s	remaining: 4.56s
489:	learn: 2.0711737	total: 22.4s	remaining: 4.52s
490:	learn: 2.0700893	total: 22.4s	remaining: 4.47s
491:	learn: 2.0699905	total: 22.4s	remaining: 4.42s
492:	learn: 2.0695900	total: 22.5s	remaining: 4.38s
493:	learn: 2.0689236	total: 22.5s	remaining: 4.33s
494:	learn: 2.0685006	total: 22.6s	remaining: 4.29s
495:	learn: 2.0684344	total: 22.6s	remaining: 4.24s
496:	learn: 2.0675962	total: 22.7s	remaining: 4.2s
497:	learn: 2.0668327	total: 22.7s	remaining: 4.15s
498:	learn: 2.0659

50:	learn: 2.6309163	total: 2.24s	remaining: 23.7s
51:	learn: 2.6266715	total: 2.29s	remaining: 23.6s
52:	learn: 2.6187929	total: 2.33s	remaining: 23.6s
53:	learn: 2.6123363	total: 2.38s	remaining: 23.6s
54:	learn: 2.6080190	total: 2.42s	remaining: 23.5s
55:	learn: 2.6041600	total: 2.47s	remaining: 23.5s
56:	learn: 2.5984819	total: 2.51s	remaining: 23.4s
57:	learn: 2.5932477	total: 2.55s	remaining: 23.4s
58:	learn: 2.5878328	total: 2.59s	remaining: 23.3s
59:	learn: 2.5826435	total: 2.63s	remaining: 23.2s
60:	learn: 2.5793314	total: 2.68s	remaining: 23.2s
61:	learn: 2.5755457	total: 2.72s	remaining: 23.1s
62:	learn: 2.5718591	total: 2.77s	remaining: 23.2s
63:	learn: 2.5686185	total: 2.82s	remaining: 23.1s
64:	learn: 2.5638401	total: 2.86s	remaining: 23.1s
65:	learn: 2.5610619	total: 2.9s	remaining: 23s
66:	learn: 2.5571594	total: 2.95s	remaining: 23s
67:	learn: 2.5549118	total: 3s	remaining: 23s
68:	learn: 2.5512798	total: 3.04s	remaining: 22.9s
69:	learn: 2.5486361	total: 3.08s	remaini

...

457:	learn: 2.3457962	total: 12s	remaining: 3.44s
458:	learn: 2.3454759	total: 12.1s	remaining: 3.42s
459:	learn: 2.3448815	total: 12.1s	remaining: 3.39s
460:	learn: 2.3448439	total: 12.1s	remaining: 3.37s
461:	learn: 2.3443147	total: 12.1s	remaining: 3.34s
462:	learn: 2.3440953	total: 12.2s	remaining: 3.31s
463:	learn: 2.3437623	total: 12.2s	remaining: 3.29s
464:	learn: 2.3430686	total: 12.2s	remaining: 3.26s
465:	learn: 2.3424652	total: 12.3s	remaining: 3.23s
466:	learn: 2.3422552	total: 12.3s	remaining: 3.21s
467:	learn: 2.3422116	total: 12.3s	remaining: 3.18s
468:	learn: 2.3417943	total: 12.3s	remaining: 3.15s
469:	learn: 2.3415811	total: 12.3s	remaining: 3.13s
470:	learn: 2.3415616	total: 12.4s	remaining: 3.1s
471:	learn: 2.3409187	total: 12.4s	remaining: 3.07s
472:	learn: 2.3407255	total: 12.4s	remaining: 3.05s
473:	learn: 2.3405061	total: 12.4s	remaining: 3.02s
474:	learn: 2.3404800	total: 12.5s	remaining: 3s
475:	learn: 2.3403126	total: 12.5s	remaining: 2.97s
476:	learn: 2.3402

28:	learn: 3.0102410	total: 941ms	remaining: 18.2s
29:	learn: 2.9780653	total: 971ms	remaining: 18.1s
30:	learn: 2.9461979	total: 999ms	remaining: 18s
31:	learn: 2.9192179	total: 1.02s	remaining: 17.8s
32:	learn: 2.8929454	total: 1.05s	remaining: 17.7s
33:	learn: 2.8689992	total: 1.08s	remaining: 17.6s
34:	learn: 2.8467122	total: 1.11s	remaining: 17.5s
35:	learn: 2.8250196	total: 1.13s	remaining: 17.4s
36:	learn: 2.8072371	total: 1.17s	remaining: 17.4s
37:	learn: 2.7886921	total: 1.19s	remaining: 17.3s
38:	learn: 2.7735741	total: 1.22s	remaining: 17.3s
39:	learn: 2.7590610	total: 1.25s	remaining: 17.2s
40:	learn: 2.7450467	total: 1.28s	remaining: 17.1s
41:	learn: 2.7325654	total: 1.31s	remaining: 17s
42:	learn: 2.7202514	total: 1.34s	remaining: 17s
43:	learn: 2.7111588	total: 1.38s	remaining: 17.1s
44:	learn: 2.7017889	total: 1.41s	remaining: 17s
45:	learn: 2.6925479	total: 1.43s	remaining: 16.9s
46:	learn: 2.6851275	total: 1.46s	remaining: 16.9s
47:	learn: 2.6761782	total: 1.49s	remai

188:	learn: 2.4634473	total: 5.68s	remaining: 12s
189:	learn: 2.4630654	total: 5.7s	remaining: 12s
190:	learn: 2.4625290	total: 5.73s	remaining: 11.9s
191:	learn: 2.4617906	total: 5.76s	remaining: 11.9s
192:	learn: 2.4607038	total: 5.79s	remaining: 11.9s
193:	learn: 2.4599017	total: 5.82s	remaining: 11.9s
194:	learn: 2.4594396	total: 5.85s	remaining: 11.8s
195:	learn: 2.4585531	total: 5.88s	remaining: 11.8s
196:	learn: 2.4578092	total: 5.91s	remaining: 11.8s
197:	learn: 2.4575465	total: 5.93s	remaining: 11.7s
198:	learn: 2.4567917	total: 5.97s	remaining: 11.7s
199:	learn: 2.4562245	total: 6.01s	remaining: 11.7s
200:	learn: 2.4557757	total: 6.04s	remaining: 11.7s
201:	learn: 2.4552620	total: 6.07s	remaining: 11.6s
202:	learn: 2.4546559	total: 6.09s	remaining: 11.6s
203:	learn: 2.4542128	total: 6.12s	remaining: 11.6s
204:	learn: 2.4534481	total: 6.15s	remaining: 11.5s
205:	learn: 2.4532783	total: 6.18s	remaining: 11.5s
206:	learn: 2.4530229	total: 6.21s	remaining: 11.5s
207:	learn: 2.452

347:	learn: 2.3985476	total: 10.3s	remaining: 7.13s
348:	learn: 2.3982115	total: 10.3s	remaining: 7.1s
349:	learn: 2.3977858	total: 10.4s	remaining: 7.07s
350:	learn: 2.3977129	total: 10.4s	remaining: 7.03s
351:	learn: 2.3971195	total: 10.4s	remaining: 7s
352:	learn: 2.3967017	total: 10.4s	remaining: 6.97s
353:	learn: 2.3962775	total: 10.5s	remaining: 6.94s
354:	learn: 2.3956552	total: 10.5s	remaining: 6.91s
355:	learn: 2.3955325	total: 10.5s	remaining: 6.88s
356:	learn: 2.3951828	total: 10.5s	remaining: 6.86s
357:	learn: 2.3948811	total: 10.6s	remaining: 6.83s
358:	learn: 2.3945586	total: 10.6s	remaining: 6.8s
359:	learn: 2.3941811	total: 10.6s	remaining: 6.77s
360:	learn: 2.3938971	total: 10.7s	remaining: 6.74s
361:	learn: 2.3934703	total: 10.7s	remaining: 6.71s
362:	learn: 2.3932109	total: 10.7s	remaining: 6.69s
363:	learn: 2.3927810	total: 10.8s	remaining: 6.66s
364:	learn: 2.3926368	total: 10.8s	remaining: 6.63s
365:	learn: 2.3922109	total: 10.8s	remaining: 6.6s
366:	learn: 2.3918

506:	learn: 2.3543108	total: 15s	remaining: 2.43s
507:	learn: 2.3541667	total: 15.1s	remaining: 2.4s
508:	learn: 2.3539597	total: 15.1s	remaining: 2.37s
509:	learn: 2.3538956	total: 15.1s	remaining: 2.35s
510:	learn: 2.3535448	total: 15.2s	remaining: 2.32s
511:	learn: 2.3534955	total: 15.2s	remaining: 2.29s
512:	learn: 2.3533920	total: 15.2s	remaining: 2.26s
513:	learn: 2.3527997	total: 15.3s	remaining: 2.23s
514:	learn: 2.3525080	total: 15.3s	remaining: 2.2s
515:	learn: 2.3522338	total: 15.3s	remaining: 2.17s
516:	learn: 2.3519960	total: 15.4s	remaining: 2.14s
517:	learn: 2.3517283	total: 15.4s	remaining: 2.11s
518:	learn: 2.3515783	total: 15.4s	remaining: 2.08s
519:	learn: 2.3512659	total: 15.5s	remaining: 2.05s
520:	learn: 2.3509604	total: 15.5s	remaining: 2.02s
521:	learn: 2.3508541	total: 15.5s	remaining: 1.99s
522:	learn: 2.3507593	total: 15.5s	remaining: 1.96s
523:	learn: 2.3505923	total: 15.6s	remaining: 1.93s
524:	learn: 2.3503564	total: 15.6s	remaining: 1.9s
525:	learn: 2.350

In [46]:
df_trial = db.select()
df_trial[df_trial['remark']=='group3 RFE2'][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
4,2019-05-18 22:57:34.706729,134,1.631104,0.000029,2.107990,0.004890,0.476886
5,2019-05-18 23:04:57.264779,114,1.628535,0.000011,2.107337,0.005743,0.478802
6,2019-05-18 23:10:27.350303,94,1.637958,0.000004,2.104357,0.005169,0.466399
7,2019-05-18 23:14:22.274333,74,1.643851,0.000050,2.111248,0.005386,0.467397
8,2019-05-18 23:16:55.341345,54,1.662980,0.000098,2.101211,0.003817,0.438231
9,2019-05-18 23:18:34.737478,34,1.700759,0.000115,2.098778,0.004548,0.398019
10,2019-05-18 23:19:36.014654,14,1.791819,0.000385,2.093661,0.004730,0.301842


In [53]:
mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.0001, 1)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':param['columns'].copy(),
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 8')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

In [57]:
for trial_i in mytrial:
    db.insert(trial_i)

In [58]:
df_trial = db.select()

In [63]:
df_trial[(df_trial['remark']=='tune 8')&(df_trial['mae_diff']<.15)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
148,2019-05-19 04:02:42.907014,134,1.978709,0.001674,2.115072,0.007527,0.136363
303,2019-05-19 04:02:42.907014,134,1.978709,0.001674,2.115072,0.007527,0.136363
157,2019-05-19 04:12:56.400131,134,1.976958,0.001654,2.117660,0.007341,0.140702
312,2019-05-19 04:12:56.400131,134,1.976958,0.001654,2.117660,0.007341,0.140702


In [51]:
db.commit()

In [48]:
len(mytrial)

155

In [36]:
# run one try
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial)

In [13]:
#  select features by permutation_weight
EP.select_features_(df_train, param, mytrial, nfeats_best=25, nfeats_removed_per_try=10, key='average_model_weight')

In [15]:
#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.0001, 1)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':param['columns'].copy(),
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
            },
            'fit':{
#                 'eval_metric':'mae',
                'verbose':False,
                'early_stopping_rounds':200,
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 437')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-16 06:40:37,822] Finished a trial resulted in value: 0.39048786322361084. Current best value is 0.39048786322361084 with parameters: {'num_trees': 443, 'depth': 8, 'learning_rate': 0.14009548720416376, 'l2_leaf_reg': 0.19118246571936606, 'bagging_temperature': 0.7249268739404343, 'random_strength': 0.10770304834217433, 'random_state': 6976}.
[I 2019-05-16 06:41:52,138] Finished a trial resulted in value: 0.4248433723672612. Current best value is 0.39048786322361084 with parameters: {'num_trees': 443, 'depth': 8, 'learning_rate': 0.14009548720416376, 'l2_leaf_reg': 0.19118246571936606, 'bagging_temperature': 0.7249268739404343, 'random_strength': 0.10770304834217433, 'random_state': 6976}.
[I 2019-05-16 06:43:04,704] Finished a trial resulted in value: 0.2728656948376179. Current best value is 0.2728656948376179 with parameters: {'num_trees': 224, 'depth': 10, 'learning_rate': 0.08599833898747268, 'l2_leaf_reg': 0.12054084484030389, 'bagging_temperature': 0.7411893498597109, 

[I 2019-05-16 07:02:44,023] Finished a trial resulted in value: 0.01758654432029695. Current best value is 0.01758654432029695 with parameters: {'num_trees': 227, 'depth': 2, 'learning_rate': 0.02558645221599814, 'l2_leaf_reg': 0.5193796128237813, 'bagging_temperature': 0.7868376900793114, 'random_strength': 0.41116064346878806, 'random_state': 3886}.
[I 2019-05-16 07:02:50,727] Finished a trial resulted in value: 0.05653289011494533. Current best value is 0.01758654432029695 with parameters: {'num_trees': 227, 'depth': 2, 'learning_rate': 0.02558645221599814, 'l2_leaf_reg': 0.5193796128237813, 'bagging_temperature': 0.7868376900793114, 'random_strength': 0.41116064346878806, 'random_state': 3886}.
[I 2019-05-16 07:02:57,212] Finished a trial resulted in value: 0.09099540961849038. Current best value is 0.01758654432029695 with parameters: {'num_trees': 227, 'depth': 2, 'learning_rate': 0.02558645221599814, 'l2_leaf_reg': 0.5193796128237813, 'bagging_temperature': 0.7868376900793114, '

[I 2019-05-16 07:12:14,900] Finished a trial resulted in value: 0.2641992973605272. Current best value is 0.010976358974680982 with parameters: {'num_trees': 205, 'depth': 4, 'learning_rate': 0.012267487045081477, 'l2_leaf_reg': 0.3129657897970629, 'bagging_temperature': 0.8197608355801962, 'random_strength': 0.5288165795250432, 'random_state': 2236}.
[I 2019-05-16 07:12:22,023] Finished a trial resulted in value: 0.21279401752645347. Current best value is 0.010976358974680982 with parameters: {'num_trees': 205, 'depth': 4, 'learning_rate': 0.012267487045081477, 'l2_leaf_reg': 0.3129657897970629, 'bagging_temperature': 0.8197608355801962, 'random_strength': 0.5288165795250432, 'random_state': 2236}.
[I 2019-05-16 07:12:31,161] Finished a trial resulted in value: 0.14516031438026997. Current best value is 0.010976358974680982 with parameters: {'num_trees': 205, 'depth': 4, 'learning_rate': 0.012267487045081477, 'l2_leaf_reg': 0.3129657897970629, 'bagging_temperature': 0.8197608355801962

[I 2019-05-16 07:22:20,517] Finished a trial resulted in value: 0.06091398142682294. Current best value is 0.010976358974680982 with parameters: {'num_trees': 205, 'depth': 4, 'learning_rate': 0.012267487045081477, 'l2_leaf_reg': 0.3129657897970629, 'bagging_temperature': 0.8197608355801962, 'random_strength': 0.5288165795250432, 'random_state': 2236}.
[I 2019-05-16 07:22:49,964] Finished a trial resulted in value: 0.3672354669182138. Current best value is 0.010976358974680982 with parameters: {'num_trees': 205, 'depth': 4, 'learning_rate': 0.012267487045081477, 'l2_leaf_reg': 0.3129657897970629, 'bagging_temperature': 0.8197608355801962, 'random_strength': 0.5288165795250432, 'random_state': 2236}.
[I 2019-05-16 07:22:56,561] Finished a trial resulted in value: 0.11774755453126651. Current best value is 0.010976358974680982 with parameters: {'num_trees': 205, 'depth': 4, 'learning_rate': 0.012267487045081477, 'l2_leaf_reg': 0.3129657897970629, 'bagging_temperature': 0.8197608355801962

[I 2019-05-16 07:30:15,108] Finished a trial resulted in value: 0.2256973907534088. Current best value is 0.007076336210938805 with parameters: {'num_trees': 200, 'depth': 3, 'learning_rate': 0.010360908233833718, 'l2_leaf_reg': 0.21177712057553388, 'bagging_temperature': 0.7691199754765087, 'random_strength': 0.720840502590669, 'random_state': 8654}.
[I 2019-05-16 07:30:19,878] Finished a trial resulted in value: 0.009371142472729152. Current best value is 0.007076336210938805 with parameters: {'num_trees': 200, 'depth': 3, 'learning_rate': 0.010360908233833718, 'l2_leaf_reg': 0.21177712057553388, 'bagging_temperature': 0.7691199754765087, 'random_strength': 0.720840502590669, 'random_state': 8654}.
[I 2019-05-16 07:30:23,926] Finished a trial resulted in value: 0.045019401056371436. Current best value is 0.007076336210938805 with parameters: {'num_trees': 200, 'depth': 3, 'learning_rate': 0.010360908233833718, 'l2_leaf_reg': 0.21177712057553388, 'bagging_temperature': 0.7691199754765

In [43]:
df_test_pred = df_trial.loc[342]['df_test_pred']

In [44]:
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission.csv', index=False)

In [20]:
df_trial = pd.DataFrame(mytrial)

In [28]:
#[df_trial['mae_diff']<.05].sort_values(by=['val_mae']) | (df_trial['remark']=='tune feats selected by group ')
df_trial[(df_trial['remark']=='tune 437')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head()

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
587,2019-05-16 07:24:47.809001,30,1.846284,0.000018,1.896202,0.000107,0.049918
452,2019-05-16 06:46:11.662876,30,1.853478,0.000005,1.900535,0.000072,0.047056
486,2019-05-16 07:02:57.205850,30,1.854923,0.000010,1.902746,0.000108,0.047823
453,2019-05-16 06:46:20.441903,30,1.855200,0.000002,1.903429,0.000057,0.048229
569,2019-05-16 07:20:58.104760,30,1.861948,0.000005,1.905226,0.000094,0.043278


In [30]:
df_trial.loc[452:452][['datetime','remark','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
452,2019-05-16 06:46:11.662876,tune 437,30,1.853478,0.000005,1.900535,0.000072,0.047056


In [29]:
df_trial.to_pickle('../trial/catboost.pkl')